In [39]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin
### fucking around
from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSamples

In [159]:
name = r'round_plant2'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (1150/1187)

In [160]:
num_points = 30
ground_truth_distance = training_data[:num_points,0]
angles = training_data[:num_points,1:3]
origins = training_data[:num_points,3:6]

In [156]:
def getSpacing(num_points, num_bins):
    """return a efewafwefwe
    """
    # TODO: add hyperparameter for tuning "slope" of inverse sigmoid function
    # create a list of magnitudes with even spacing from 0 to 1
    t = torch.linspace(0,1, num_bins).expand(num_points, num_bins)  # [batch_size, num_bins//2]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins//2]

    # apply inverse sigmoid function to even spacing
    t = torch.log(t / (1 - t))  
    t = rearrange(t, 'a b -> (a b) 1')  # [num_bins, batch_size]  transpose for multiplication broadcast
    return t  

def getSamples(centres, angles, r, num_bins = 100):
    num_points = r.shape[0]
    elev = angles[:,0]
    pan = angles[:,1]
    x_tilde = cos(elev)*cos(pan)      
    y_tilde = cos(elev)*sin(pan)      
    z_tilde = sin(elev)
    unit_vectors = torch.tensor([x_tilde, y_tilde, z_tilde])

    # process vectors: [3, num_points] -> [num_points*num_bins, 3]
    unit_vectors_repeated = repeat(unit_vectors, 'c n -> (n b) c', b = num_bins)
    r_repeated = repeat(r, 'n -> (n b) 1', b = num_bins)
    t = getSpacing(num_points, num_bins)
    sample_magnitudes = t + r_repeated
    pos = unit_vectors_repeated*sample_magnitudes      # [num_bins*num_points, 3]

    # tile the origin values
    centres_tiled = torch.tensor(repeat(centres, 'n c -> (n b) c', b = num_bins)) # [num_bin*batch_size, 3]
    # complete getting sample position by adding camera centre position to sampled position
    pos = centres_tiled + pos

    # tile the angle too
    angles_tiled = torch.tensor(repeat(angles, 'n c -> (n b) c', b = num_bins))
    return pos, angles_tiled, centres_tiled


In [161]:
sample_pos, sample_ang, sample_org = getSamples(origins, angles, ground_truth_distance, num_bins=100)
# sample_pos = getSamples(origins, angles, ground_truth_distance, num_bins=100)


In [164]:
sample_org

tensor([[ 9.2510, -5.3099, -0.3636],
        [ 9.2510, -5.3099, -0.3636],
        [ 9.2510, -5.3099, -0.3636],
        ...,
        [ 5.1147, -5.1280, -0.0550],
        [ 5.1147, -5.1280, -0.0550],
        [ 5.1147, -5.1280, -0.0550]], dtype=torch.float64)

In [162]:
# Convert the numpy array to an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(sample_pos)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])